In [1]:
import os
import pandas as pd
import config

In [16]:
delivs_path = os.path.join(config.base_dir, 'AllDeliveries05192021.csv')

In [17]:
delivs_df = pd.read_csv(delivs_path)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
delivs_df['Delivery Date'] = pd.to_datetime(delivs_df['Delivery Date'])
delivs_df['Member ID'] = delivs_df['Member ID'].astype(str)

In [19]:
generated_pm_path = os.path.join(config.base_dir, '05-18-2021', 'PM', 'AllDeliveriesPM.csv')
generated_am_path = os.path.join(config.base_dir, '05-18-2021', 'AM', 'AllDeliveriesAM.csv')

In [20]:
generated_pm_df = pd.read_csv(generated_pm_path)
generated_pm_df['Member ID'] = generated_pm_df['Member ID'].astype(str)

generated_am_df = pd.read_csv(generated_am_path)
generated_am_df['Member ID'] = generated_am_df['Member ID'].astype(str)

In [21]:
def filt_for_day(df, day):
    df["Delivery Date"] = df["Delivery Date"].apply(lambda x: x.date())
    filt = df["Delivery Date"] == day
    return df[filt]

def filt_for_future(df):
    future_filt = df["Delivery Status"] == 'Future'
    pick_up_filt = df["Delivery Status"] == "Future - Pick up"
    remaining_filt = ~(future_filt | pick_up_filt)
    if not df[remaining_filt].empty:
        print("The following entries were filtered out.")
        print(df[remaining_filt][["MainContact", "Member ID", "Delivery Status"]])
    return df[future_filt], df[pick_up_filt]

def split_am_pm(df):
    #Split the days deliveries into morning and afternoon.
    am_filt = df["DeliveryTime"].str.upper() == "AM"
    pm_filt = df["DeliveryTime"].str.upper() == "PM"

    left_overs = df[~am_filt & ~pm_filt] 
    if not left_overs.empty:
        print("The following entries have no delivery time:")
        print(left_overs[["MainContact", "Member ID", "Delivery Status"]])

    am_df = df[am_filt]
    pm_df = df[pm_filt]

    return am_df, pm_df

In [22]:
main_day_df = filt_for_day(delivs_df, pd.Timestamp('2021-05-18'))

In [23]:
hard_copy_am_df, hard_copy_pm_df = split_am_pm(main_day_df)

In [24]:
all(hard_copy_pm_df['Delivery Status'] == 'Future')

True

In [25]:
filt_pm = hard_copy_pm_df['Member ID'].isin(generated_pm_df['Member ID'])
filt_am = hard_copy_am_df['Member ID'].isin(generated_am_df['Member ID'])

In [26]:
am_missed = hard_copy_am_df[~filt_am]
pm_missed = hard_copy_pm_df[~filt_pm]

all_missed = pd.concat([am_missed, pm_missed], axis=0)

In [27]:
prev_filt = delivs_df['Delivery Date'] <= pd.Timestamp('2021-05-17')

In [28]:
earliers = delivs_df[prev_filt]

In [29]:
forgotten_filt = earliers['Delivery Status'] == 'Future'

In [30]:
set(earliers[forgotten_filt]['Delivery Date'])

{datetime.date(2021, 5, 4),
 datetime.date(2021, 5, 7),
 datetime.date(2021, 5, 13),
 datetime.date(2021, 5, 17)}

In [31]:
all_missed.to_csv('all_missed.csv')